In [1]:
import pandas
import numpy
import os
import dateutil.parser as date_parser
from datetime import datetime
import functools

## Vaccination Data [[Link](https://data.cdc.gov/Vaccinations/COVID-19-Vaccination-Trends-in-the-United-States-N/rh2h-3yt2/)]

In [2]:
vaccination_data = pandas.read_csv('https://data.cdc.gov/api/views/rh2h-3yt2/rows.csv?accessType=DOWNLOAD')
vaccination_data.rename({'Location': 'state', 'Date': 'date'}, axis=1, inplace=True)
vaccination_data.date = vaccination_data.date.apply(lambda x: str(date_parser.parse(x).date()))
vaccination_data = vaccination_data[vaccination_data.date_type == 'Report']

## COVID Outcomes (cases and deaths) [[Link](https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36/data)]

In [3]:
covid_data = pandas.read_csv('https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD')
covid_data.rename({'submission_date': 'date'}, axis=1, inplace=True)
covid_data.date = covid_data.date.apply(lambda x: str(date_parser.parse(x).date()))

In [4]:
merged_df = functools.reduce(lambda a, b: pandas.merge(a, b, on=['date', 'state'], how='outer'), [vaccination_data, covid_data])
merged_df = merged_df.sort_values(by=['date', 'state'])

In [6]:
merged_df.tail()

,date,MMWR_week,state,Administered_Daily,Administered_Cumulative,Administered_7_Day_Rolling_Average,Admin_Dose_1_Daily,Admin_Dose_1_Cumulative,Admin_Dose_1_Day_Rolling_Average,date_type,...,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
25,2022-01-07,1.0,VT,3398.0,1269972.0,2376.0,720.0,568866.0,507.0,Report,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-07,1.0,WA,36997.0,12777537.0,22509.0,7145.0,5763632.0,4342.0,Report,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,2022-01-07,1.0,WI,24111.0,9264594.0,17209.0,5062.0,4021302.0,3493.0,Report,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2022-01-07,1.0,WV,5521.0,2522852.0,3795.0,1228.0,1121235.0,894.0,Report,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2022-01-07,1.0,WY,103.0,691672.0,787.0,53.0,328094.0,225.0,Report,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
merged_df.to_csv('covid_outcomes.csv', index=False)